# 実験の再現性を検証する

Sapporoを用いて実施された実験の再現性を検証し、その結果をレポートします。   
このタスクで検証されるのは以下の3点です。  
- RO-Crate 内の値に基づいて、ワークフローが再実行可能であること
- 再実行のステータスが、RO-Crate に記載されているステータスと同一であること
- 再実行で得られた結果ファイルのサイズ・チェックサムが、RO-Crate の値と同一であること
 
検証結果は必ず同期する必要はありません。この環境で検証結果を確認いただいたあとに、GIN-forkへ同期するか破棄するか選択いただけます。  
検証結果を同期する場合は、研究リポジトリの直下の**validation_results**フォルダに格納されます。タスク実行後に検証結果を確認する際はvalidation_resultsフォルダ配下の**results.json**をご確認ください。  
同期しない場合は、検証結果は破棄されます。

研究フロートップページに戻る場合は[こちら](../../base_FLOW.ipynb)。新規タブで開きます。  

## 0. 研究リポジトリ名を確認する

以下のセルを実行すると、この実験実行環境で操作する研究リポジトリ名を確認できます。 

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
utils.show_name('green')

## 1. 再現性を確認したい実験パッケージを選択する

以下のセルを実行して、再現性を検証したい実験パッケージを選んでください。  
選択を間違えた場合は、このセルを再実行してください。

In [ ]:
import json
from ipywidgets import Dropdown, Button
from IPython.display import clear_output

# 実験パッケージ名を取得
with open('/home/jovyan/pipeline.json', 'r') as f:
    pipeline = json.load(f)

# 再現する実験パッケージを選択
def on_click_callback(clicked_button: Button) -> None:
    global package
    package=dropdown.value
    clear_output()
    print("入力を受けつけました：", package)

dropdown = Dropdown(
    options=pipeline,
    description='実験パッケージ名:',
    disabled=False,
)

button = Button(description='入力完了')
button.on_click(on_click_callback)
display(dropdown, button)

## 2. 研究データを最新化する

以下のセルを実行すると、この実験実行環境で操作する研究リポジトリ名を確認できます。 

In [ ]:
import os

package_dir = "/home/jovyan/experiments/" + package
result_dir = package_dir + "/output_data"

!datalad update -s gin --merge
os.chdir(package_dir)
!datalad get .

## 3. 再現性に関わるメタデータを検証する

In [ ]:
import os
import json
import subprocess

cmd = ["python3", "/home/jovyan/nii-dg/sapporo_example/package_ro_crate.py", "http://dg02.dg.rcos.nii.ac.jp:3000", result_dir]
result = subprocess.run(cmd, capture_output=True, text=True)
if result.returncode == 0:
    print("メタデータをパッケージングしました。")
    !cat ro-crate-metadata.json
else:
    print("メタデータのパッケージングに失敗しました。")
    print(result.stderr)

In [ ]:
from urllib import parse
import requests
import os
import json
import logging

with open("/home/jovyan/experiments/test1/ro-crate-metadata.json") as f:
    ro_crate = json.load(f)

try:
    # generate url_for_request_validation
    url_for_request_validation = parse.urlunparse((
        'http',
        'dg02.dg.rcos.nii.ac.jp:443', 
        'validate',
        '',
        '',
        ''
    ))
    # request validation
    headers = {'content-type': 'application/json'}
    response = requests.post(url_for_request_validation, data=json.dumps(ro_crate), headers=headers)
    result = response.json()
except Exception as e:
    logging.error('検証サービスに接続できません。')
    logging.error(str(e))
else:
    if response.status_code == requests.codes.ok:
        os.chdir(os.environ['HOME'])
        from WORKFLOWS.utils.tmp_validation import tmp_validation
        request_id = result['request_id']
        tmp_validation.save_request_id(request_id)
        print('正常に検証が依頼されました。次の処理にお進みください。\nリクエストID：' + request_id)
    elif response.ok == False:
        logging.error('異常が発生しました。担当者にお問い合わせください。')
        logging.error(result['message'])

## 4. 検証結果を確認する

In [ ]:
import os
import json
import requests
from urllib import parse
import time
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from WORKFLOWS.utils.tmp_validation import tmp_validation
import logging

def on_click_callback(clicked_button: widgets.Button) -> None:
    clear_output()
    print('検証結果の出力を削除しました。\n再度確認したい場合は、次のセルを実行する前にこのセルを再実行してください。')

def show_results():
    logging.error('検証の結果、メタデータに以下の不備が見つかりました。確認後、「確認を完了する」ボタンをクリックして次にお進みください。'\
                  '\n「確認を完了する」ボタンがクリックされていない場合は、検証結果を含んだこのノートブックがリポジトリに同期されます。')
    button = widgets.Button(description='確認を完了する')
    button.on_click(on_click_callback)
    !cat $result_file_path
    display(button)
    
request_id = tmp_validation.get_request_id()
counter = 5
while counter >0:
    try:
        # generate url_for_get_validation_results
        url_for_get_validation_results = parse.urlunparse((
            'http',
            'dg02.dg.rcos.nii.ac.jp:443', 
            request_id,
            '',
            '',
            ''
        ))
        # request get validation results
        response = requests.get(url_for_get_validation_results)
        result = response.json()
        counter -= 1
        clear_output()
    except Exception as e:
        logging.error('検証サービスに接続できません。')
        logging.error(str(e))
        break
    else:
        if response.status_code == requests.codes.ok:
            status = result['status']
            if status == 'UNKNOWN':
                logging.error('リクエストID：' + request_id + 'の状況が読み込めませんでした。')
                break
            elif any([status == 'QUEUED', status == 'RUNNING']):
                print('リクエストID：' + request_id + 'は検証完了していません。時間をおいて再度確認します。')
                time.sleep(5)
                continue
            elif status == 'COMPLETE':
                tmp_validation.save_verification_results(result)
                print('すべてのメタデータは適切に管理されています。次にお進みください。')
                break
            elif status == 'FAILED':
                tmp_validation.save_verification_results(result)
                result_file_path = tmp_validation.fetch_validation_results_file_path()
                show_results()
                break
            elif status == 'EXECUTOR_ERROR':
                logging.error('正常に検証を実行できませんでした。')
                break
            elif status == 'CANCELING':
                print('リクエストID：' + request_id + 'は現在キャンセル中です。')
                break
            elif status == 'CANCELED':
                print('リクエストID：' + request_id + 'はキャンセルされました。')
                break

        elif response.ok == False:
            logging.error('異常が発生しました。担当者にお問い合わせください。')
            logging.error(result['message'])
                      
else:
    clear_output()
    logging.error('検証に時間がかかっています。時間をおいて再度このセルを実行してください。')

## 5. 研究リポジトリに同期する

このタスクの実行結果を研究リポジトリに同期します。  
検証結果を同期するか破棄するかは、「5.1. 検証結果を研究リポジトリに同期するか選択する」で選択できます。

### 5.1. 検証結果を研究リポジトリに同期するか破棄するかを選択する

In [ ]:
import panel as pn

pn.extension()
column = pn.Column()

def save_selection_result(event):
    global need_sync
    done_button.button_type = "success"
    done_button.name = "選択完了しました。次の処理にお進みください。"
    need_sync = True if select.value == 1 else False

select = pn.widgets.Select(name='検証結果を同期するか破棄するかを選択した後、完了ボタンをクリックしてください。', options={'同期する': 1, '同期せずに破棄する':2})
done_button = pn.widgets.Button(name= "選択を完了する", button_type= "primary")
done_button.on_click(save_selection_result)
column.append(select)
column.append(done_button)
column

### 5.2. 研究リポジトリに同期する

研究リポジトリにこのタスクの実行結果を同期します。  
「5.1. 検証結果を研究リポジトリに同期するか破棄するかを選択する」で同期せずに破棄するを選択した場合は、検証結果は同期されずこのファイルの実行結果のみが同期されます。

In [ ]:
import os
from IPython.display import display, Javascript
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
os.chdir(os.environ['HOME'])
from WORKFLOWS.utils.tmp_validation import tmp_validation

tmp_validation.operate_validation_results(need_sync)
git_path = ['/home/jovyan/WORKFLOWS/FLOW/03_after_experiments_phase/base_validate_repeatability.ipynb']
if need_sync == True:
    validation_result_path = tmp_validation.fetch_validation_result_path()
    git_path.append(validation_result_path)

display(Javascript('IPython.notebook.save_checkpoint();'))
is_ok = utils.syncs_with_repo(git_path=git_path, gitannex_path=None, gitannex_files=None, message='実験パッケージ' + package + 'の再現性検証')
if is_ok:
    tmp_validation.delete_verification_results_and_request_id()

## 6. 研究フロートップページに遷移する

以下のセルを実行し、表示されるリンクをクリックして研究フロートップページに戻ってください。

In [ ]:
from IPython.display import display, HTML, Javascript
display(HTML("<a href='../../base_FLOW.ipynb'>研究フロートップページに遷移する</a>"))
display(Javascript('IPython.notebook.save_checkpoint();'))